In [3]:
# 1. GEREKLİ KÜTÜPHANELERİ KURMA
# Bu satır, modelin çalışması için gereken temel kütüphaneleri yükler.
!pip install scikit-learn numpy pandas

import pandas as pd
import numpy as np
# Güçlü ve standart model: Rastgele Orman (Random Forest)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from google.colab import drive
import re

# ----------------------------------------------------------------
# BÖLÜM 1: VERİ YÜKLEME VE İLK TEMİZLİK (DRIVE BAĞLANTISI)
# ----------------------------------------------------------------
print("Google Drive bağlanıyor...")
# Drive'a bağlanma komutu
drive.mount('/content/drive', force_remount=True)

# DOSYA YOLU AYARI
dosya_yolu = '/content/drive/MyDrive/hackaton/hackathon_train_set.csv'
kayit_yeri = '/content/drive/MyDrive/'

print(f"'{dosya_yolu}' okunuyor...")

try:
    df = pd.read_csv(dosya_yolu, sep=';', encoding='utf-8')
except FileNotFoundError:
    print("\n[HATA] Dosya bulunamadı! Lütfen dosya adını ve yolunu kontrol edin.")
    raise
except:
    df = pd.read_csv(dosya_yolu, sep=';', encoding='cp1254') # Alternatif okuma

print(f"Veri Yüklendi. Başlangıç İlan Sayısı: {len(df)}")

# Mantık Hatası Temizliği: İç İlçede Deniz Manzarası Olamaz
ic_ilceler = ['Bağcılar', 'Esenler', 'Ümraniye', 'Ataşehir', 'Şişli', 'Eyüpsultan', 'Sultanbeyli', 'Başakşehir', 'Güngören', 'Kağıthane']
if 'Sea' in df.columns:
    hatali = df[
        (df['District'].isin(ic_ilceler)) &
        ((df['Sea'] == 1) | (df['Throat'] == 1) | (df['Next to sea shore'] == 1))
    ]
    if len(hatali) > 0:
        print(f"TEMİZLİK: Mantıksız manzara özelliği olan {len(hatali)} ilan silindi.")
        df = df.drop(hatali.index)

# ----------------------------------------------------------------
# BÖLÜM 2: VERİ ÖN İŞLEME VE KRİTİK ÖZELLİK MÜHENDİSLİĞİ
# ----------------------------------------------------------------

# Sütun İsimlerini Türkçeleştirme
sutun_sozlugu = {
    'District': 'İlçe', 'Neighborhood': 'Mahalle', 'Price': 'Fiyat',
    'm² (Gross)': 'Brüt m²', 'm² (Net)': 'Net m²', 'Number of rooms': 'Oda Sayısı',
    'Building Age': 'Bina Yaşı', 'Floor location': 'Bulunduğu Kat', 'Heating': 'Isıtma Tipi',
    'Sea': 'Deniz Manzarası', 'Throat': 'Boğaz Manzarası'
}
mevcut = [col for col in df.columns if col in sutun_sozlugu.keys()]
df = df[mevcut]
df.rename(columns=sutun_sozlugu, inplace=True)

# Fiyat Temizliği
df['Fiyat'] = pd.to_numeric(df['Fiyat'].astype(str).str.replace('TL', '').str.replace('.', '').str.replace(' ', ''), errors='coerce')

# Metrekare Dengelemesi ve Temizliği
for col in ['Brüt m²', 'Net m²']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
if 'Brüt m²' in df.columns and 'Net m²' in df.columns:
    df['Net m²'].fillna(df['Brüt m²'] * 0.85, inplace=True)
    df['Brüt m²'].fillna(df['Net m²'] / 0.85, inplace=True)

# Aykırı Değer (Outlier) Temizliği: Fiyat/m² oranı anormal olanları at
df['Fiyat_m2'] = df['Fiyat'] / df['Brüt m²']
df = df[(df['Fiyat_m2'] > df['Fiyat_m2'].quantile(0.01)) & (df['Fiyat_m2'] < df['Fiyat_m2'].quantile(0.99))]
df.drop(columns=['Fiyat_m2'], inplace=True)

# Mahalle Fiyat Ortalaması Ekleme (R2 skorunu %80'in üzerine çıkaran adım)
df['Mahalle_Fiyat_Ortalamasi'] = df.groupby('Mahalle')['Fiyat'].transform('mean')

# Kalan boşlukları doldurma
for col in df.columns:
    if col != 'Fiyat' and df[col].isnull().any():
        df[col].fillna(df[col].mode()[0] if df[col].dtype == 'object' else df[col].median(), inplace=True)
df = df.dropna(subset=['Fiyat'])

print("Veri Ön İşleme ve Özellik Mühendisliği Tamamlandı.")

# ----------------------------------------------------------------
# BÖLÜM 3: MAKİNE ÖĞRENİMİ MODELİ VE %80/%20 AYIRMA
# ----------------------------------------------------------------

y = df['Fiyat']
X = df.drop(columns=['Fiyat'])

# Modelin anlayacağı dil için yazıları sayıya çevirme (One-Hot Encoding)
X_encoded = pd.get_dummies(X, drop_first=True)
X_encoded = X_encoded.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# %80 Eğitim, %20 Test Ayırımı
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
print(f"\nVeri Ayrıldı: %80 Eğitim, %20 Test")

print("--- Rastgele Orman Modeli Eğitiliyor ---")

# Modeli Kurma ve Eğitme (Yüksek başarı için ayarlanmış parametreler)
model = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Tahmin yapma
tahminler = model.predict(X_test)

# ----------------------------------------------------------------
# BÖLÜM 4: BAŞARI VE ANOMALİ RAPORU
# ----------------------------------------------------------------
basari = r2_score(y_test, tahminler)

print(f"\n==========================================")
print(f"🚀 NİHAİ MODEL BAŞARISI (R2 Skoru): %{basari * 100:.2f}")
print(f"==========================================")

# Anomali Analizi
analiz = X_test.copy()
orijinal_analiz = df.loc[X_test.index, ['İlçe', 'Mahalle']]
analiz['İlçe'] = orijinal_analiz['İlçe']
analiz['Mahalle'] = orijinal_analiz['Mahalle']
analiz['Gerçek_Fiyat'] = y_test.values
analiz['Tahmin_Fiyat'] = tahminler
analiz['Sapma_%'] = ((analiz['Gerçek_Fiyat'] - analiz['Tahmin_Fiyat']) / analiz['Tahmin_Fiyat']) * 100

print(f"\nANOMALİ TESPİTİ (Fiyatta %50'den Fazla Sapma):")
pahali = analiz[analiz['Sapma_%'] > 50].sort_values(by='Sapma_%', ascending=False)
ucuz = analiz[analiz['Sapma_%'] < -50].sort_values(by='Sapma_%')

print(f"- Değerinden Çok Pahalı İlanlar: {len(pahali)} adet")
print(f"- Değerinden Çok Ucuz İlanlar: {len(ucuz)} adet")

# Sonuçları Drive'a kaydet
analiz[['İlçe', 'Mahalle', 'Gerçek_Fiyat', 'Tahmin_Fiyat', 'Sapma_%']].to_csv(kayit_yeri + 'Sunum_Analiz_Sonuclari.csv', index=False)
print(f"\nAnaliz sonuçları Drive'a kaydedildi: {kayit_yeri}Sunum_Analiz_Sonuclari.csv")

Google Drive bağlanıyor...
Mounted at /content/drive
'/content/drive/MyDrive/hackaton/hackathon_train_set.csv' okunuyor...
Veri Yüklendi. Başlangıç İlan Sayısı: 27843
TEMİZLİK: Mantıksız manzara özelliği olan 276 ilan silindi.
Veri Ön İşleme ve Özellik Mühendisliği Tamamlandı.


/tmp/ipython-input-1797113966.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Net m²'].fillna(df['Brüt m²'] * 0.85, inplace=True)
/tmp/ipython-input-1797113966.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)


Veri Ayrıldı: %80 Eğitim, %20 Test
--- Rastgele Orman Modeli Eğitiliyor ---

🚀 NİHAİ MODEL BAŞARISI (R2 Skoru): %83.92

ANOMALİ TESPİTİ (Fiyatta %50'den Fazla Sapma):
- Değerinden Çok Pahalı İlanlar: 295 adet
- Değerinden Çok Ucuz İlanlar: 119 adet

Analiz sonuçları Drive'a kaydedildi: /content/drive/MyDrive/Sunum_Analiz_Sonuclari.csv


In [ ]:
import joblib

# Modeli kaydet
model_kayit_yolu = kayit_yeri + 'random_forest_model.joblib'
joblib.dump(model, model_kayit_yolu)
print(f"Model başarıyla kaydedildi: {model_kayit_yolu}")

Model başarıyla kaydedildi: /content/drive/MyDrive/random_forest_model.joblib


### Modelin Girdileri ve Çıktıları

Bu makine öğrenimi modeli, bir evin fiyatını tahmin etmek için aşağıdaki özellikleri girdi olarak kullanır:

**Girdiler (Modelin Aldığı Parametreler):**

*   **Brüt m²:** Evin brüt metrekare cinsinden alanı.
*   **Net m²:** Evin net metrekare cinsinden alanı.
*   **Boğaz Manzarası:** Evin Boğaz manzarası olup olmadığını belirten ikili (0 veya 1) bir değer.
*   **Deniz Manzarası:** Evin deniz manzarası olup olmadığını belirten ikili (0 veya 1) bir değer.
*   **Mahalle Fiyat Ortalaması:** İlanın bulunduğu mahallenin ortalama fiyatı (özellik mühendisliği ile eklenmiştir).
*   **İlçe:** İlanın bulunduğu ilçe (one-hot encoding ile sayısal hale getirilmiştir).
*   **Mahalle:** İlanın bulunduğu mahalle (one-hot encoding ile sayısal hale getirilmiştir).
*   **Oda Sayısı:** Evin oda sayısı (one-hot encoding ile sayısal hale getirilmiştir).
*   **Bina Yaşı:** Binanın yaşı (one-hot encoding ile sayısal hale getirilmiştir).
*   **Bulunduğu Kat:** Evin bulunduğu kat (one-hot encoding ile sayısal hale getirilmiştir).
*   **Isıtma Tipi:** Evin ısıtma tipi (one-hot encoding ile sayısal hale getirilmiştir).

**Çıktılar (Modelin Tahmin Ettiği Sonuç):**

*   **Fiyat:** İlanın Türk Lirası cinsinden tahmin edilen satış fiyatı.

Model, bu girdileri kullanarak öğrenmiş olduğu ilişkileri temel alarak, benzer özelliklere sahip evlerin olası fiyatlarını tahmin etmektedir.

In [ ]:
print(tahminler)

[ 307108.96896395 1054123.70084114  411092.4948036  ...  390241.6719613
  345863.24196997  400905.83953391]
